<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/1-Introduction%20to%20LLMs%20with%20OpenAI/1_2-Easy_NL2SQL_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<h1>Large Language Models Projects</h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>1.2-Create a simple Natural Language to SQL with OpenAI and Gradio.</h2>
    
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)
<hr>

Models: gpt-3.5-turbo / gpt-4o-mini

Colab Environment: CPU

Keys:
* NL2SQL
* Gradio
* Code Generation
* Prompt Hardening.

Article related: [Create a Natural Language to SQL Solution with OpenAI and Gradio](https://pub.towardsai.net/first-nl2sql-chat-with-openai-and-gradio-b1de0d6541b4?sk=7e0346b93130e70574645d5d390adfe9)

# SQL Generator
A sample of how to build a translator from natural language to SQL:

* GPT 35 / gpt-4o-mini
* OpenAI


In [ ]:
!pip install -q openai==1.1.1
!pip install -q gradio==4.41.0

In [ ]:
import openai
import panel as pn
from getpass import getpass
import gradio as gr

In [ ]:
openai.api_key=getpass("OpenAI API Key: ")
#model = "gpt-3.5-turbo"
model = "gpt-4o-mini"

In [ ]:
context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with
this is your SQL, and after that an SQL that can do what the user request.

Your SQL Database is composed by some tables.
Try to Maintain the SQL order simple.
Just after the SQL add a simple and concise text explaining how it works.
If the user ask for something that can not be answered with information from the DB
just answer something nice and simple, maximum 10 words, asking him for a new question that
can be solved with SQL.
"""} ]

context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "string"
    }
  ]
}
"""
})

context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "string"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "string"
    }
  ]
}
"""
})

In [ ]:
def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [ ]:
#The function that Gradio will use.
def gradio_chat(message, history):
    #Add the instructions to the prompt.
    history_chat = context

    #Add the history that Gradio send to us.
    for user, assistant in history:
        history_chat.append({"role":"user", "content":user})
        history_chat.append({"role":"assistant", "content":assistant})

    #Add the las user message.
    history_chat.append({"role":"user", "content":message})
    history_chat.append({'role':'system', 'content':f"""Only return SQL Orders.
     If you can't return and SQL order, say sorry, and ask, politely but concisely, for a new question."""})

    #Call OpenAI and return the response.
    return continue_conversation(history_chat, 0)

In [ ]:
#Customized gradio textbox.
InputText = gr.Textbox(label="order", info="Talk with DB", scale= 6)
examples=["Who is the highest-paid employee?", "How many employes with degrees do we have?"]

In [ ]:
gr.ChatInterface(gradio_chat,
                 textbox=InputText,
                 retry_btn=None,
                 undo_btn=None,
                 title="SQL Generator",
                 examples=examples,
                submit_btn="Get My SQL").launch()